<a href="https://colab.research.google.com/github/dsfdev2023/studentEvaluatiuon/blob/main/HR_SL_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit plotly pandas openpyxl pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.8/731.8 kB 41.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-7.0.1-py3-none-any.whl size=21122 sha256=4b21592d64324806a616106e2f5eade7c45844e77b431eafcf09151082fe5bd4
  Stored in directory: /root/.cache/pip/wheels/3b/32/0e/27789b6fde02bf2b320d6f1a0fd9e1354b257c5f75eefc29bc
Successfully built pyngrok


In [12]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import plotly.express as px

# Function to convert time to minutes
def convert_to_minutes(time_val):
    if pd.isnull(time_val):
        return 0
    if isinstance(time_val, int):
        return time_val
    elif isinstance(time_val, str):
        hours, minutes = map(int, time_val.split(':'))
        return hours * 60 + minutes
    elif hasattr(time_val, 'hour'):
        return time_val.hour * 60 + time_val.minute
    else:
        return 0

# Streamlit app
def main():
    st.title('HR Data Visualizations')

    # File uploader allows user to add their own Excel
    uploaded_file = st.file_uploader("Upload your input Excel file", type=["xlsx"])

    if uploaded_file is not None:
        try:
            df = pd.read_excel(uploaded_file)

            # Data validation
            expected_columns = ['Ligne', 'real Presence', 'Breaks', 'Pause']
            if not all(column in df.columns for column in expected_columns):
                st.error("Uploaded file does not contain the required columns.")
                return

            # Preprocess the DataFrame
            df['real Presence'] = df['real Presence'].apply(convert_to_minutes)
            df['Pause'] = df['Pause'].apply(convert_to_minutes)

            # Perform aggregations
            grouped = df.groupby('Ligne').agg({
                'real Presence': 'sum',
                'Breaks': 'sum',
                'Pause': 'sum'
            }).reset_index()

            # Create Plotly visualizations
            fig_real_presence = px.bar(grouped, x='Ligne', y='real Presence', color='Ligne',
                                labels={'real Presence': 'Real Presence (Minutes)'},
                                title='Sum of Real Presence by Ligne')
            fig_real_presence.update_layout(showlegend=False)

            fig_breaks = px.bar(grouped, x='Ligne', y='Breaks', color='Ligne',
                            labels={'Breaks': 'Breaks (Count)'},
                            title='Sum of Breaks by Ligne')
            fig_breaks.update_layout(showlegend=False)

            fig_pause = px.bar(grouped, x='Ligne', y='Pause', color='Ligne',
                            labels={'Pause': 'Pause (Minutes)'},
                            title='Sum of Pause by Ligne')
            fig_pause.update_layout(showlegend=False)

            # Display the Plotly visualizations in Streamlit
            st.plotly_chart(fig_real_presence, use_container_width=True)
            st.plotly_chart(fig_breaks, use_container_width=True)
            st.plotly_chart(fig_pause, use_container_width=True)

        except Exception as e:
            st.error(f"An error occurred: {e}")

# Run the main function
if __name__ == "__main__":
    main()


Overwriting streamlit_app.py


In [3]:
import os

# Download and unzip ngrok only if it doesn't exist yet
if not os.path.exists("ngrok-stable-linux-amd64.zip"):
    !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
    !unzip -o ngrok-stable-linux-amd64.zip

# Authenticate with ngrok using the new auth token
!./ngrok authtoken 2WVfogA5KBfaWWOutV2RTCL2oPe_3Fwruktu2FJTakjgArzMG


--2023-11-20 17:10:10--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  49.5MB/s    in 0.3s    

2023-11-20 17:10:10 (49.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [13]:
# Start ngrok for the Streamlit port 8501
get_ipython().system_raw('./ngrok http 8501 &')
!sleep 2  # Wait a couple of seconds for the tunnel to be established

# Fetch and print the public URL
public_url = !curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; tunnels = json.load(sys.stdin)['tunnels']; print(tunnels[0]['public_url'] if tunnels else '')"
print(public_url[0])


https://4a04-34-73-47-238.ngrok-free.app


In [ ]:
# Run the Streamlit app
!streamlit run streamlit_app.py &





  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.73.47.238:8501

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
